# The Product Pricer Continued

A model that can estimate how much something costs, from its description.

## AT LAST - it's time for Fine Tuning!

After all this data preparation, and old school machine learning, we've finally arrived at the moment you've been waiting for. Fine-tuning a model.

In [1]:
# Import necessary libraries

# Standard libraries
import os             # For interacting with the operating system (file paths, environment variables)
import re             # For regular expression operations
import math           # For mathematical functions
import json           # For working with JSON data
import random         # For generating random numbers and shuffling data

# Environment variable management
from dotenv import load_dotenv  # For loading environment variables from a .env file

# Hugging Face and AI integration
from huggingface_hub import login  # For interacting with Hugging Face Hub (authentication, datasets)
from items import Item             # Presumably imports the 'Item' class from a local file

# Data visualization and analysis
import matplotlib.pyplot as plt     # For creating plots and visualizations
import numpy as np                 # For numerical computations and array manipulations
import pickle                      # For serializing and deserializing Python objects

# Collections module for counting and managing data
from collections import Counter    # For counting hashable objects (e.g., word counts)

# OpenAI and Anthropic for AI model interaction
from openai import OpenAI           # For OpenAI's API integration (e.g., GPT models)
from anthropic import Anthropic    # For Anthropic's API integration (e.g., Claude models)


In [2]:
# Import the Tester class from the testing module
from testing import Tester

In [3]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [2]:
openai = OpenAI()

In [6]:
%matplotlib inline

In [ ]:
# Let's avoid curating all our data again! Load in the pickle files:

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [8]:
# OpenAI recommends fine-tuning with populations of 50-100 examples
# But as our examples are very small, I'm suggesting we go with 200 examples (and 1 epoch)

fine_tune_train = train[:200]
fine_tune_validation = train[200:250]

# Step 1

Prepare our data for fine-tuning in JSONL (JSON Lines) format and upload to OpenAI

In [9]:
# First let's work on a good prompt for a Frontier model
# Notice that I'm removing the " to the nearest dollar"
# When we train our own models, we'll need to make the problem as easy as possible, 
# but a Frontier model needs no such simplification.

def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": f"Price is ${item.price:.2f}"}
    ]

In [ ]:
messages_for(train[0])

In [11]:
# Convert the items into a list of json objects - a "jsonl" string
# Each row represents a message in the form:
# {"messages" : [{"role": "system", "content": "You estimate prices...


def make_jsonl(items):
    result = ""
    for item in items:
        messages = messages_for(item)
        messages_str = json.dumps(messages)
        result += '{"messages": ' + messages_str +'}\n'
    return result.strip()

In [ ]:
# Print the result of calling make_jsonl() on the first 3 items of the training set

# Assuming make_jsonl() is a function that converts data into a JSONL (JSON Lines) format,
# the following line processes the first 3 items from the 'train' dataset and prints the result
print(make_jsonl(train[:3]))

In [13]:
# Convert the items into jsonl and write them to a file

def write_jsonl(items, filename):
    with open(filename, "w") as f:
        jsonl = make_jsonl(items)
        f.write(jsonl)

In [14]:
write_jsonl(fine_tune_train, "fine_tune_train.jsonl")

In [15]:
write_jsonl(fine_tune_validation, "fine_tune_validation.jsonl")

In [16]:
# Open the fine-tune training file in binary mode and upload it to OpenAI for fine-tuning purposes
with open("fine_tune_train.jsonl", "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

In [ ]:
train_file

In [18]:
with open("fine_tune_validation.jsonl", "rb") as f:
    validation_file = openai.files.create(file=f, purpose="fine-tune")

In [ ]:
validation_file

# Step 2

I love Weights and Biases - a beautiful, free platform for monitoring training runs.  
Weights and Biases is integrated with OpenAI for fine-tuning.

First set up your weights & biases free account at:

https://wandb.ai

From the Avatar >> Settings menu, near the bottom, you can create an API key.

Then visit the OpenAI dashboard at:

https://platform.openai.com/account/organization

In the integrations section, you can add your Weights & Biases key.

## And now time to Fine-tune!

In [20]:
# Define a dictionary for integrating with Weights & Biases (wandb)

# 'wandb_integration' is a dictionary that sets up the integration with Weights & Biases (wandb)
# It specifies the type of integration and the configuration for the 'wandb' project
wandb_integration = {"type": "wandb", "wandb": {"project": "gpt-pricer"}}

In [ ]:
# Access the 'id' attribute of the 'train_file'
train_file.id

In [ ]:
# Create a fine-tuning job with OpenAI's API

# Call the 'openai.fine_tuning.jobs.create' method to create a fine-tuning job
openai.fine_tuning.jobs.create(
    # Specify the training file by passing the 'id' of the 'train_file'
    training_file=train_file.id,
    
    # Specify the validation file by passing the 'id' of the 'validation_file'
    validation_file=validation_file.id,
    
    # Define the base model to fine-tune, in this case, a specific variant of GPT-4
    model="gpt-4o-mini-2024-07-18",
    
    # Set a random seed for reproducibility of results
    seed=42,
    
    # Specify hyperparameters for the fine-tuning job (e.g., number of epochs)
    hyperparameters={"n_epochs": 1},
    
    # Specify integration with Weights & Biases for tracking the experiment
    integrations=[wandb_integration],
    
    # Define a suffix to append to the fine-tuned model's name for easier identification
    suffix="pricer"
)


In [ ]:
# List fine-tuning jobs with a limit of 1 result

# Call the 'openai.fine_tuning.jobs.list' method to retrieve fine-tuning jobs
# The 'limit=1' argument restricts the result to just 1 fine-tuning job
openai.fine_tuning.jobs.list(limit=1)

In [24]:
# Get the ID of the most recent fine-tuning job

# Call 'openai.fine_tuning.jobs.list' to retrieve the list of fine-tuning jobs (limit to 1 job)
# Extract the ID of the first (most recent) job from the response and assign it to 'job_id'
job_id = openai.fine_tuning.jobs.list(limit=1).data[0].id

In [ ]:
job_id

In [ ]:
# Retrieve details of a specific fine-tuning job by its ID

# Call the 'openai.fine_tuning.jobs.retrieve' method to get detailed information about a fine-tuning job
# Use 'job_id' to specify which job's details you want to retrieve
openai.fine_tuning.jobs.retrieve(job_id)

In [ ]:
# Retrieve the events of a specific fine-tuning job

# Call 'openai.fine_tuning.jobs.list_events' to get the events associated with the fine-tuning job
# Use 'job_id' to specify which fine-tuning job's events you want to retrieve
# The 'limit=10' argument restricts the response to the latest 10 events
openai.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10).data


# Step 3

Test our fine tuned model

In [28]:
# Retrieve the name of the fine-tuned model from a specific fine-tuning job

# Call 'openai.fine_tuning.jobs.retrieve' to get the details of the fine-tuning job using its 'job_id'
# Extract the 'fine_tuned_model' attribute, which contains the name of the fine-tuned model
fine_tuned_model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model

In [29]:
fine_tuned_model_name

In [ ]:
print(fine_tuned_model_name)

In [3]:
def list_fine_tuning_jobs():
    try:
        # List all fine-tuning jobs
        jobs = openai.fine_tuning.jobs.list(limit=10)  # Adjust the limit as needed
        
        if jobs.data:
            # Print the job IDs and their status
            for job in jobs.data:
                print(f"Job ID: {job.id} | Status: {job.status}")
        else:
            print("No fine-tuning jobs found.")
    
    except Exception as e:
        print(f"Error: {str(e)}")

# Call the function to list jobs
list_fine_tuning_jobs()

Job ID: ftjob-1mmrRAhkGxLMrKpzfFhcwrQA | Status: succeeded
Job ID: ftjob-OS7G69K5OY4Zt9xn2iRmWtJL | Status: succeeded


In [ ]:
job_status = openai.fine_tuning.jobs.retrieve("job name")
fine_tuned_model_name = job_status.fine_tuned_model
print(f"Fine-tuned model name: {fine_tuned_model_name}")


In [ ]:
fine_tuned_model_name = "Model"

In [30]:
# The prompt

def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [ ]:
# Try this out

messages_for(test[0])

In [32]:
# A utility function to extract the price from a string

def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [ ]:
get_price("The price is roughly $99.99 because blah blah")

In [34]:
# The function for gpt-4o-mini

def gpt_fine_tuned(item):
    response = openai.chat.completions.create(
        model=fine_tuned_model_name, 
        messages=messages_for(item),
        seed=42,
        max_tokens=7
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
print(test[0].price)
print(gpt_fine_tuned(test[0]))

In [ ]:
print(test[0].test_prompt())

In [ ]:
Tester.test(gpt_fine_tuned, test)